<a href="https://colab.research.google.com/github/RodrigoEslava/Keras/blob/main/Deep_Learning_para_Bioinform%C3%A1tica_deep_learning_com_Keras_parte_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando dados

In [ ]:
!wget -O membrane.fasta 'https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28%28taxonomy_id%3A2%29%20AND%20%28cc_scl_term%3ASL-0162%29%20AND%20%28reviewed%3Atrue%29%29'

--2023-03-09 23:08:04--  https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28%28taxonomy_id%3A2%29%20AND%20%28cc_scl_term%3ASL-0162%29%20AND%20%28reviewed%3Atrue%29%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘membrane.fasta’

membrane.fasta          [               <=>  ]  23.14M  3.10MB/s    in 10s     

2023-03-09 23:08:16 (2.21 MB/s) - ‘membrane.fasta’ saved [24260077]



In [ ]:
!wget -O cytoplasm.fasta 'https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28%28taxonomy_id%3A2%29%20AND%20%28cc_scl_term%3ASL-0086%29%20AND%20%28reviewed%3Atrue%29%29'

--2023-03-09 23:08:16--  https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28%28taxonomy_id%3A2%29%20AND%20%28cc_scl_term%3ASL-0086%29%20AND%20%28reviewed%3Atrue%29%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘cytoplasm.fasta’

cytoplasm.fasta         [                 <=>]  56.33M  2.30MB/s    in 26s     

2023-03-09 23:08:43 (2.15 MB/s) - ‘cytoplasm.fasta’ saved [59070211]



In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. Pre-processamento

In [ ]:
from Bio import SeqIO
import pandas as pd

rows = []

for record in SeqIO.parse('membrane.fasta', 'fasta'):
  rows.append({'sequence': str(record.seq), 'location': 'membrane'})

for record in SeqIO.parse('cytoplasm.fasta', 'fasta'):
  rows.append({'sequence': str(record.seq), 'location': 'cytoplasm'})

df = pd.DataFrame(rows)

In [ ]:
df.groupby(by=['location']).count()

,sequence
location,
cytoplasm,120252
membrane,49634


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler()
df_resampled, _ = sampler.fit_resample(df, df['location'])

In [ ]:
df_resampled.groupby(by=['location']).count()

,sequence
location,
cytoplasm,49634
membrane,49634


In [ ]:
df_resampled = df_resampled.sample(30000)

In [ ]:
df_resampled.groupby(by=['location']).count()

,sequence
location,
cytoplasm,15076
membrane,14924


In [ ]:
X = df_resampled['sequence']
y = df_resampled['location']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from tensorflow import keras

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(X_train)

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens  = tokenizer.texts_to_sequences(X_test)

In [ ]:
MAX_SEQ_LEN = max(X_train.map(len))
MAX_SEQ_LEN

10624

In [ ]:
X_train_tokens_padded = keras.preprocessing.sequence.pad_sequences(
    X_train_tokens, 
    maxlen=1500
)

X_test_tokens_padded = keras.preprocessing.sequence.pad_sequences(
    X_test_tokens, 
    maxlen=1500
)

In [ ]:
X_test_tokens_padded.shape

(7500, 1500)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y_train)

y_train_encoded = keras.utils.to_categorical(le.transform(y_train), num_classes=2)
y_test_encoded  = keras.utils.to_categorical(le.transform(y_test), num_classes=2)

y_train_encoded

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

# 3. Treinamento de modelo

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(len(tokenizer.word_index)+1, 20, input_length=1500))
model.add(keras.layers.Conv1D(32, 8))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1500, 20)          460       
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 32)          5152      
                                                                 
 flatten_2 (Flatten)         (None, 47776)             0         
                                                                 
 dense_4 (Dense)             (None, 32)                1528864   
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,534,542
Trainable params: 1,534,542
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(
    X_train_tokens_padded, 
    y_train_encoded, 
    validation_data=(X_test_tokens_padded, y_test_encoded),
    epochs=10,
    batch_size=32
  )

Epoch 1/10
704/704 [==============================] - 7s 8ms/step - loss: 0.3870 - acc: 0.8156 - val_loss: 0.3279 - val_acc: 0.8587
Epoch 2/10
704/704 [==============================] - 4s 6ms/step - loss: 0.2084 - acc: 0.9172 - val_loss: 0.2345 - val_acc: 0.9084
Epoch 3/10
704/704 [==============================] - 4s 6ms/step - loss: 0.1193 - acc: 0.9559 - val_loss: 0.2264 - val_acc: 0.9205
Epoch 4/10
704/704 [==============================] - 5s 7ms/step - loss: 0.0622 - acc: 0.9795 - val_loss: 0.2544 - val_acc: 0.9199
Epoch 5/10
704/704 [==============================] - 4s 6ms/step - loss: 0.0343 - acc: 0.9888 - val_loss: 0.3183 - val_acc: 0.9136
Epoch 6/10
704/704 [==============================] - 5s 7ms/step - loss: 0.0228 - acc: 0.9918 - val_loss: 0.3098 - val_acc: 0.9177
Epoch 7/10
704/704 [==============================] - 4s 6ms/step - loss: 0.0350 - acc: 0.9873 - val_loss: 0.3244 - val_acc: 0.9203
Epoch 8/10
704/704 [==============================] - 4s 6ms/step - loss: 0.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(len(tokenizer.word_index)+1, 20, input_length=1500))
model.add(keras.layers.Conv1D(64, 8))
model.add(keras.layers.Conv1D(32, 8))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(
    X_train_tokens_padded, 
    y_train_encoded, 
    validation_data=(X_test_tokens_padded, y_test_encoded),
    epochs=10,
    batch_size=32
  )

Epoch 1/10
704/704 [==============================] - 11s 12ms/step - loss: 0.5204 - acc: 0.7293 - val_loss: 0.4259 - val_acc: 0.8151
Epoch 2/10
704/704 [==============================] - 7s 11ms/step - loss: 0.4142 - acc: 0.8136 - val_loss: 0.3396 - val_acc: 0.8512
Epoch 3/10
704/704 [==============================] - 6s 9ms/step - loss: 0.3435 - acc: 0.8531 - val_loss: 0.2861 - val_acc: 0.8791
Epoch 4/10
704/704 [==============================] - 7s 10ms/step - loss: 0.3001 - acc: 0.8764 - val_loss: 0.2678 - val_acc: 0.8971
Epoch 5/10
704/704 [==============================] - 6s 9ms/step - loss: 0.2635 - acc: 0.8944 - val_loss: 0.2599 - val_acc: 0.8961
Epoch 6/10
704/704 [==============================] - 7s 10ms/step - loss: 0.2442 - acc: 0.9014 - val_loss: 0.2333 - val_acc: 0.9139
Epoch 7/10
704/704 [==============================] - 7s 9ms/step - loss: 0.2172 - acc: 0.9163 - val_loss: 0.2237 - val_acc: 0.9040
Epoch 8/10
704/704 [==============================] - 7s 10ms/step - lo

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(len(tokenizer.word_index)+1, 20, input_length=1500))
model.add(keras.layers.LSTM(32))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(
    X_train_tokens_padded, 
    y_train_encoded, 
    validation_data=(X_test_tokens_padded, y_test_encoded),
    epochs=10,
    batch_size=32
  )

Epoch 1/10
704/704 [==============================] - 35s 44ms/step - loss: 0.6451 - acc: 0.5876 - val_loss: 0.5248 - val_acc: 0.7076
Epoch 2/10
704/704 [==============================] - 32s 45ms/step - loss: 0.5383 - acc: 0.7224 - val_loss: 0.5417 - val_acc: 0.7100
Epoch 3/10
704/704 [==============================] - 31s 44ms/step - loss: 0.4772 - acc: 0.8011 - val_loss: 0.4232 - val_acc: 0.8045
Epoch 4/10
704/704 [==============================] - 31s 44ms/step - loss: 0.5992 - acc: 0.6307 - val_loss: 0.6927 - val_acc: 0.5212
Epoch 5/10
704/704 [==============================] - 30s 42ms/step - loss: 0.6903 - acc: 0.5232 - val_loss: 0.6812 - val_acc: 0.5587
Epoch 6/10
704/704 [==============================] - 31s 45ms/step - loss: 0.6665 - acc: 0.5827 - val_loss: 0.6952 - val_acc: 0.4973
Epoch 7/10
704/704 [==============================] - 29s 42ms/step - loss: 0.6928 - acc: 0.5218 - val_loss: 0.6937 - val_acc: 0.4973
Epoch 8/10
704/704 [==============================] - 32s 46ms

In [ ]:
import numpy as np

def predict_location(protein_sequence):
  sequences = [protein_sequence]
  sequences_tokens = tokenizer.texts_to_sequences(sequences)
  sequences_tokens_padded = keras.utils.pad_sequences(sequences_tokens, maxlen=1500)
  y_pred = np.argmax(model.predict(sequences_tokens_padded, verbose=False), axis=1)
  return le.inverse_transform(y_pred)[0]

predict_location('MWSSRTV')

'membrane'

In [ ]:
for record in SeqIO.parse('membrane.fasta', 'fasta'):
  print(record.id, predict_location(str(record.seq)))

sp|A0A089QRB9|MSL3_MYCTU membrane
sp|A0A0H2URG7|GTFA_STRPN cytoplasm
sp|A0A0H2V871|IROE_ECOL6 membrane
sp|A0A0H2V8B5|TCPC_ECOL6 membrane
sp|A0A0H2VG78|GLCP_STAES membrane
sp|A0A0H2ZMF9|PBP2A_STRP2 membrane
sp|A0A0H3GDH9|PGDA_LISM4 membrane
sp|A0A0H3GGY3|PGPH_LISM4 membrane
sp|A0A0H3M5A8|PPMNT_MYCBP membrane
sp|A0A0K8P8E7|MHETH_IDESA membrane
sp|A0A1S7LCW6|MAMP_MAGMO cytoplasm
sp|A0A2S3R7M0|MARTX_VIBVL membrane
sp|A0A2S4N3N0|OMPA_SHIFL cytoplasm
sp|A0A2T4VDM4|GSDM_VITXG membrane
sp|A0A3Q0NBH7|PGDA_LISMG membrane
sp|A0A3S5YBC7|EGCSE_RHOH1 cytoplasm
sp|A0LNN5|SFMCT_SYNFM membrane
sp|A0QNG1|PKNB_MYCS2 cytoplasm
sp|A0QP27|MMPL3_MYCS2 membrane
sp|A0QQF4|TTFA_MYCS2 cytoplasm
sp|A0QR29|MSPA_MYCS2 membrane
sp|A0QVH8|RIP1_MYCS2 membrane
sp|A0QWG5|ACYLT_MYCS2 membrane
sp|A0QWG6|PIMA_MYCS2 membrane
sp|A0QZ13|LNT_MYCS2 membrane
sp|A0R1E8|PKS5_MYCS2 membrane
sp|A1C3L9|GTFA_STRPA cytoplasm
sp|A1C3M0|GTFB_STRPA cytoplasm
sp|A1JUB7|YADA_YERE8 membrane
sp|A1YKW7|RTXA_KINKI membrane
sp|A5A616|MGTS_ECOLI 

KeyboardInterrupt: ignored

In [ ]:
model.save('model.h5')

In [ ]:
load_model = keras.models.load_model('model.h5')
load_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1500, 20)          460       
                                                                 
 conv1d_4 (Conv1D)           (None, 1493, 64)          10304     
                                                                 
 conv1d_5 (Conv1D)           (None, 1486, 32)          16416     
                                                                 
 flatten_4 (Flatten)         (None, 47552)             0         
                                                                 
 dropout_2 (Dropout)         (None, 47552)             0         
                                                                 
 dense_8 (Dense)             (None, 64)                3043392   
                                                                 
 dropout_3 (Dropout)         (None, 64)               